In [1]:
import numpy as np
import os
os.getcwd()

data = np.load("../data/processed/supervised_train.npz")
X, y = data["X"], data["y"]

print("X shape:", X.shape)          # Expect (N, 12, 8, 8)
print("y shape:", y.shape)          # Expect (N,)
print("dtype:", X.dtype, y.dtype)
print("action_space:", y.max() + 1, "max index:", y.max())

X shape: (348113, 12, 8, 8)
y shape: (348113,)
dtype: float32 int64
action_space: 4661 max index: 4660


In [ ]:
import os, sys, numpy as np, torch
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd() / "src"))

from chess.action_encoding import ACTION_SIZE
from chess.state_encoding import board_to_tensor
from chess.ChessGame import ChessGame
from chess_nnet.NNetWrapper import NNetWrapper
from core.MCTS import MCTS
from envs.chess_gym import GymChessEnv

def basic_dataset_checks(npz_path="data/processed/supervised_train.npz"):
    assert os.path.exists(npz_path), f"Missing {npz_path}"
    data = np.load(npz_path)
    X, y = data["X"], data["y"]
    print("X.shape", X.shape, "dtype", X.dtype)
    print("y.shape", y.shape, "dtype", y.dtype)
    print("min/max y:", int(y.min()), int(y.max()))
    assert X.ndim == 4 and X.shape[1:] == (12,8,8)
    assert y.ndim == 1 and len(y) == X.shape[0]
    assert np.isfinite(X).all(), "NaN/Inf in X"
    assert np.isfinite(y).all(), "NaN/Inf in y"
    assert (y >= 0).all() and (y < ACTION_SIZE).all(), "labels out of range"
    print("unique label count:", np.unique(y).size, "/", ACTION_SIZE)

def model_and_mcts_smoke():
    game = ChessGame()
    nnet = NNetWrapper(game)
    # small forward pass (create dummy canonical board)
    sample_board = game.getInitBoard()
    canon = game.getCanonicalForm(sample_board, 1)
    board_tensor = board_to_tensor(canon).astype(np.float32)
    pi, v = nnet.predict(board_tensor)
    print("predict -> pi.shape", np.shape(pi), "v", v)
    assert np.shape(pi)[0] == ACTION_SIZE
    mcts = MCTS(game, nnet, type("Args", (), {"numMCTSSims":10, "cpuct":1.0})())
    probs = mcts.getActionProb(canon, temp=1)
    print("mcts probs shape", np.shape(probs), "sum", np.sum(probs))
    assert np.isclose(np.sum(probs), 1, atol=1e-6)

def gym_smoke():
    env = GymChessEnv()
    obs, info = env.reset()
    print("env.reset obs shape", np.array(obs).shape)
    # get valid moves from wrapped Game
    board = env._board
    player = env._player
    valid = env._game.getValidMoves(board, player)
    act = int(np.argmax(valid)) if valid.sum()>0 else 0
    obs2, reward, term, trunc, info = env.step(act)
    print("env.step -> reward", reward, "term", term, "trunc", trunc)

if __name__ == "__main__":
    basic_dataset_checks()
    model_and_mcts_smoke()
    gym_smoke()

ModuleNotFoundError: No module named 'chess_nnet'